In [37]:
# Generate reduced dataset to work with Weka

import pandas as pd
import numpy as np


data = pd.read_csv('data/x_train_gr_smpl.csv')
labels = pd.read_csv('data/y_train_smpl.csv')
data['label'] = labels

In [38]:
data['label'] = data['label'].map({0: 'zero', 1: 'one', 2: 'two', 3: 'three', 4: 'four', 5: 'five', 6: 'six', 7: 'seven', 8: 'eight', 9: 'nine'})
data

In [40]:
data = data.sample(frac=1).reset_index(drop=True)
data[:8000].to_csv('data/reduce_dataset.csv', encoding='utf-8', sep=',', index=False)